Assignment วิเคราะห์รีวิว 1000 รีวิว

In [74]:
!pip install -U langchain-ollama langchain-openai

In [75]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [76]:
!ls /content/drive/MyDrive/review_dataset

w_review_train.csv  w_review_train.parquet


In [77]:
import duckdb

con = duckdb.connect(database=':memory:', read_only=False)

In [78]:
df = con.execute("SELECT * FROM read_csv_auto('/content/drive/MyDrive/review_dataset/w_review_train.csv', parallel=false, encoding='UTF-8', ignore_errors=true)").fetchdf()

df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5


In [79]:
df.shape

(40000, 2)

In [80]:
con.execute("COPY df TO '/content/drive/MyDrive/review_dataset/w_review_train.parquet' (FORMAT PARQUET)")

In [81]:
parquet_avg_rating_direct = con.execute("SELECT AVG(column1) FROM '/content/drive/MyDrive/review_dataset/w_review_train.parquet' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [82]:
parquet_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.parquet'

In [83]:
parquet_avg_rating_direct = con.execute(f"SELECT AVG(column1) FROM '{parquet_file_path}' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [84]:
coffee_keywords = [
    'กาแฟ', 'คาเฟ่', 'coffee', 'cafe', 'เอสเปรสโซ่', 'ลาเต้',
    'มอคค่า', 'คาปูชิโน่', 'เฟรนช์เพรส', 'cold brew', 'drip',
    'espresso', 'latte', 'americano', 'macchiato', 'brew',
    'specialty', 'slow bar', 'ร้านกาแฟ', 'ร้านคาเฟ่', 'กาแฟสด',
    'เมนูกาแฟ', 'กาแฟนม', 'กาแฟดำ', 'ร้านบาริสต้า', 'barista', 'coffee shop'
]

where_clause = " OR ".join([f"column0 LIKE '%{keyword}%'" for keyword in coffee_keywords])

In [85]:
%%time
coffee_reviews_df = con.execute(f"SELECT * FROM '{parquet_file_path}' WHERE {where_clause}").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 2.33 s, sys: 36.3 ms, total: 2.37 s
Wall time: 3.29 s


In [86]:
csv_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.csv'

In [87]:
%%time
coffee_reviews_df_csv = con.execute(f"SELECT * FROM read_csv_auto('{csv_file_path}', parallel=false, encoding='UTF-8', ignore_errors=true) WHERE {where_clause}").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 2.61 s, sys: 55.1 ms, total: 2.66 s
Wall time: 4.18 s


In [88]:
coffee_reviews_df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...,4
2,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...,3
3,ร้านอาหารญี่ปุ่นร้านนี้ ใจจริงไม่อยากแนะนำเลยค...,5
4,เดือนแรกที่เค้าต่อคิวกัน 2 - 3 ชั่วโมง เราก็ว่...,5


In [89]:
coffee_reviews_df.shape

(4981, 2)

In [90]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [91]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [92]:
!ollama pull gemma3:270m

In [93]:
!ollama list

NAME                                  ID              SIZE      MODIFIED               
gemma3:270m                           e7d36fb2c3b3    291 MB    Less than a second ago    
scb10x/typhoon2.1-gemma3-4b:latest    8cfab9097c9d    2.6 GB    4 minutes ago             


In [94]:
!ollama run gemma3:270m "สวัสดี"

สวัสดีค่ะ 😊



In [95]:
!curl -s http://127.0.0.1:11434/api/tags

{"models":[{"name":"gemma3:270m","model":"gemma3:270m","modified_at":"2025-10-20T17:12:54.210394265Z","size":291554930,"digest":"e7d36fb2c3b3293cfe56d55889867a064b3a2b22e98335f2e6e8a387e081d6be","details":{"parent_model":"","format":"gguf","family":"gemma3","families":["gemma3"],"parameter_size":"268.10M","quantization_level":"Q8_0"}},{"name":"scb10x/typhoon2.1-gemma3-4b:latest","model":"scb10x/typhoon2.1-gemma3-4b:latest","modified_at":"2025-10-20T17:08:38.863005099Z","size":2615662352,"digest":"8cfab9097c9d142fa16c7556826c9a7874f4642ac6d575e2d66a4f4979c48b76","details":{"parent_model":"","format":"gguf","family":"gemma3","families":["gemma3"],"parameter_size":"3.9B","quantization_level":"Q4_K_M"}}]}

In [96]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [97]:
llm = OllamaLLM(model="gemma3:270m")

In [98]:
response = llm.invoke("สวัสดี")
print(response)

สวัสดีค่ะ 😊



In [99]:
template = """
คุณเป็นครูสอน Python
คำถาม : {question}
"""

In [100]:
prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

In [101]:
formatted_prompt = prompt.format(question="เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10")
print(formatted_prompt)


คุณเป็นครูสอน Python
คำถาม : เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10



In [102]:
chain = prompt | llm

In [103]:
response = chain.invoke({"question": "เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10"})

In [104]:
response

'แน่นอนครับ! ผมจะช่วยคุณเขียนโค้ด Python เพื่อหาผลรวมของตัวเลข 1 ถึง 10 โดยใช้ไลบริสต์ `numbers`\n\nนี่คือโค้ด Python ที่จะใช้ไลบริสต์ `numbers` เพื่อหาผลรวมของตัวเลข 1 ถึง 10:\n\n```python\ndef find_sum_of_numbers(numbers):\n    """\n    หาผลรวมของตัวเลข 1 ถึง 10\n    """\n    try:\n        if not numbers:\n            return 0  # ถ้าไม่มีตัวเลข 1-10 ให้หาผลรวม 0\n        total = 0\n        for number in numbers:\n            total += number\n        return total\n    except Exception as e:\n        print(f"ข้อผิดพลาด: {e}")\n        return 0  # ถ้าเกิดข้อผิดพลาด ให้ return 0\n\n# ตัวอย่างการใช้งาน\nnumbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\nresult = find_sum_of_numbers(numbers)\nprint(result)\n```\n\nโค้ดนี้จะทำงานดังนี้:\n\n1.  **`find_sum_of_numbers(numbers)`:**\n    *   ใช้ `numbers` เป็นไลบริสต์เพื่อเก็บตัวเลข 1 ถึง 10\n    *   ใช้ `try...except` เพื่อจัดการกับกรณีที่ตัวเลข 1-10 ไม่พบ\n    *   ตรวจสอบว่า `numbers` มีขนาดเป็น 0\n    *   ถ้าไม่พบตัวเลข 1-10 จะ return 0\n    *   เมื

In [105]:
from IPython.display import display, Markdown

In [106]:
display(Markdown(response))

แน่นอนครับ! ผมจะช่วยคุณเขียนโค้ด Python เพื่อหาผลรวมของตัวเลข 1 ถึง 10 โดยใช้ไลบริสต์ `numbers`

นี่คือโค้ด Python ที่จะใช้ไลบริสต์ `numbers` เพื่อหาผลรวมของตัวเลข 1 ถึง 10:

```python
def find_sum_of_numbers(numbers):
    """
    หาผลรวมของตัวเลข 1 ถึง 10
    """
    try:
        if not numbers:
            return 0  # ถ้าไม่มีตัวเลข 1-10 ให้หาผลรวม 0
        total = 0
        for number in numbers:
            total += number
        return total
    except Exception as e:
        print(f"ข้อผิดพลาด: {e}")
        return 0  # ถ้าเกิดข้อผิดพลาด ให้ return 0

# ตัวอย่างการใช้งาน
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = find_sum_of_numbers(numbers)
print(result)
```

โค้ดนี้จะทำงานดังนี้:

1.  **`find_sum_of_numbers(numbers)`:**
    *   ใช้ `numbers` เป็นไลบริสต์เพื่อเก็บตัวเลข 1 ถึง 10
    *   ใช้ `try...except` เพื่อจัดการกับกรณีที่ตัวเลข 1-10 ไม่พบ
    *   ตรวจสอบว่า `numbers` มีขนาดเป็น 0
    *   ถ้าไม่พบตัวเลข 1-10 จะ return 0
    *   เมื่อเรียก `find_sum_of_numbers` จะทำการหาผลรวมของตัวเลข 1 ถึง 10 และคืนค่าผลรวม
    *   ถ้าเกิดข้อผิดพลาด (เช่น ตัวเลข 1-10 ไม่พบ) จะคืนค่า 0

**คำอธิบายโค้ด:**

*   `numbers`: เป็นไลบริสต์ที่เก็บตัวเลข 1 ถึง 10
*   `find_sum_of_numbers(numbers)`: เป็นฟังก์ชันที่รับตัวเลข 1 ถึง 10 เป็นค่าเริ่มต้น
*   `try...except`: บังคับให้ฟังก์ชันทำงานหากมีการเกิดข้อผิดพลาด
*   `total = 0`: กำหนดค่า `total` เป็น 0 เพื่อเก็บผลรวม
*   `for number in numbers:`: กำหนดตัวเลข 1 ถึง 10 เป็นรายการ
*   `total += number`: เพิ่มค่า `number` เข้าไปใน `total`
*   `return total`:  คืนค่า `total`
*   `except Exception as e:`: บังคับให้ฟังก์ชันทำงานหากเกิดข้อผิดพลาด
*   `print(f"ข้อผิดพลาด: {e}")`: แสดงข้อผิดพลาดให้ผู้ใช้เข้าใจ

**การใช้งาน:**

1.  คุณสามารถเรียก `find_sum_of_numbers()` เพื่อหาผลรวมของตัวเลข 1 ถึง 10
2.  เมื่อคุณเรียกฟังก์ชันนี้ คุณจะได้รับผลลัพธ์ที่แสดงผลรวมของตัวเลข 1 ถึง 10
3.  คุณสามารถปรับเปลี่ยนค่า `numbers`  ให้เหมาะสมกับความต้องการของคุณได้

**ตัวอย่างการใช้งาน:**

```python
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = find_sum_of_numbers(numbers)
print(result)
```

จะแสดงผลลัพธ์ดังนี้:

```
15
```

**ข้อดีของโค้ดนี้:**

*   **ใช้งานง่าย:**  โค้ดนี้ทำได้ง่ายและเข้าใจง่าย
*   **สามารถปรับแต่งได้:** คุณสามารถปรับแต่งค่า `numbers`  ให้เหมาะสมกับความต้องการของคุณได้
*   **สามารถทำงานกับตัวเลขจำนวนมาก:**  โค้ดนี้สามารถทำงานกับตัวเลขจำนวนมากได้

หวังว่าโค้ดนี้จะเป็นประโยชน์! หากมีคำถามเพิ่มเติม ถามได้เลยครับ

In [107]:
prompt1 = PromptTemplate(
    template="แปลโจทย์เกี่ยวกับ {question}"
)

chain1 = prompt1 | llm

In [108]:
prompt2 = PromptTemplate(
    template="ทำความเข้าใจโจทย์ที่แปลแล้วต่อไปนี้ {translate}"
)

chain2 = prompt2 | llm

In [109]:
prompt3 = PromptTemplate(
    template="แก้โจทย์ตามที่เข้าใจต่อไปนี้ {understand}"
)

chain3 = prompt3 | llm

In [110]:
prompt4 = PromptTemplate(
    template="อธิบายวิธิีทำจากผลลัทธ์ต่อไปนี้ {solve}"
)

chain4 = prompt4 | llm

In [111]:
parser_chain = StrOutputParser()

In [112]:
full_chain = chain1 | chain2 | chain3 | chain4 | parser_chain

In [113]:
result = full_chain.invoke({"question":"2x + 5 = 15"})

In [114]:
result

'ถูกต้องครับ โจทย์ที่แปลแล้วคือ:\n\n**2x + 5 = 15**\n\nคำตอบคือ **2x = 10**'

In [115]:
import os
from langchain_openai import ChatOpenAI

In [116]:
OPENAI_API_KEY = "xxx"

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [117]:
llm2 = ChatOpenAI(model="gpt-4o-mini")

In [118]:
chain1 = prompt1 | llm2
chain2 = prompt2 | llm2
chain3 = prompt3 | llm2
chain4 = prompt4 | llm2

full_chain = chain1 | chain2 | chain3 | chain4 | parser_chain

In [119]:
formatted_result = result.replace('\\[', '$$') \
                         .replace('\\]', '$$') \
                         .replace('\\(', '$') \
                         .replace('\\)', '$')

In [120]:
display(Markdown(formatted_result))

ถูกต้องครับ โจทย์ที่แปลแล้วคือ:

**2x + 5 = 15**

คำตอบคือ **2x = 10**

In [121]:
prompt_template = """
[1. Role/Context บอก LLM ว่าควรทำตัวเป็นอะไร]
คุณเป็น AI ที่เชี่ยวชาญในการวิเคราะห์รีวิวภาษาไทย

[2. Task Description บอกงานที่ต้องทำ]
จาก Review ต่อไปนี้ ให้ Extract 3 keywords
ที่เป็นตัวแทนของ Review นี้ได้ดีที่สุด

[3. Input Data ข้อมูลที่ต้องประมวลผล]
Review: {review}

[4. Output Format บอกรูปแบบของคำตอบ]
ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'keywords'
เช่น {{"keywords": "กาแฟ, บรรยากาศ, ราคา"}

[5. Constraints ข้อจำกัดหรือเงื่อนไข]
- ต้องเป็นคำภาษาไทย
- คั่นด้วย comma
- หากไม่พบให้ตอบ 'ไม่ระบุ'

[6. Examples (optional) ตัวอย่าง input/output]
ตัวอย่าง
Input: "กาแฟอร่อย บรรยากาศดี ราคาแพง"
Output: {"keywords": "กาแฟ, บรรยากาศ, ราคา"}
"""

In [122]:
prompt = """
Extract keywords from this review
{review}
"""

# Pros Simple, fast
# Cons อาจได้ผลไม่ดีกับ Tasks ซับซ้อน

In [123]:
prompt = """
Extract keywords from reviews.

Example 1
Review: "กาแฟอร่อย บริการดี"
Keywords: กาแฟ, บริการ

Example 2
Review: "ราคาแพง แต่คุ้มค่า"
Keywords: ราคา, คุณภาพ

Now extract from this review
Review: {review}
Keywords:
"""

# Pros ได้ผลดีกว่า Zero-shot
# Cons Prompt ใช้ Tokens มากขึ้น

In [124]:
prompt = """
Review: {review}

ขั้นตอนการ Extract keywords
1. อ่านรีวิวและเข้าใจความหมาย
2. ระบุหัวข้อหลักที่กล่าวถึง
3. เลือก 3 keywords ที่สำคัญที่สุด
4. ตอบในรูปแบบ JSON: {{"keywords": "คำ1, คำ2, คำ3"}}

ให้ทำทีละขั้นตอน
"""

# Pros ได้ผลดีกับ Reasoning tasks
# Cons ใช้ Tokens มากกว่า ช้ากว่า

In [125]:
keyword_prompt = PromptTemplate(
template = """
จาก Review ต่อไปนี้
Extract 3 keywords ที่เป็นตัวแทนของ Review นี้

Review: {review}

ขั้นตอนการ Extract keywords
1. อ่านรีวิวและเข้าใจความหมาย
2. ระบุหัวข้อหลักที่กล่าวถึง
3. เลือก 3 keywords ที่สำคัญที่สุด
4. ตอบในรูปแบบ JSON โดยมี key คือ keyword เช่น :{{"keywords": "คำ1, คำ2, คำ3"}}

ให้ทำทีละขั้นตอน
""")

In [126]:
sample_reviews = coffee_reviews_df['column0'].head(2)
sample_reviews

,column0
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...


In [127]:
sample_reviews.to_csv('/content/sample_reviews.csv', index=False)

In [128]:
keyword_extract = keyword_prompt | llm | StrOutputParser()

In [129]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 43.4 ms, sys: 2.98 ms, total: 46.3 ms
Wall time: 3.73 s


In [130]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text ```json
{
  "keywords": [
    "ร้านอาหารใหญ่",
    "ห้องน้ำ",
    "กาแฟ",
    "น้ำผึ้ง",
    "ไข่มัน",
    "คาว",
    "ประทับใจ",
    "ราคา",
    "เกินไป",
    "รับไม่ไหว",
    "ว"
  ]
}
```

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text ```json
"keywords": ["ลาเต้", "กาแฟ", "ร้านวาวี", "อารีย์", "Starbuck", "แฟนๆ", "Starbuck", "โปรด", "test", "Taste", "อร่อย", "ราคาไม่แพง", "กินบ่อย", "จุกได้", "ถ่ายรูป", "ซากอารยธรรม", "เมพพพพ", "เมพพ", "กาแฟ", "รสชาติ", "กลิ่น", "มวยรุ่นเดียวกะ Starbuck", "การชิม", "รสชาติและกลิ่น", "เมนูอื่นๆ", "สรุป", "ถ่ายโดย", "Tan"
```



In [131]:
keyword_extract = keyword_prompt | llm2 | StrOutputParser()

In [132]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text ```json
{
  "keywords": [
    "ร้านอาหารใหญ่",
    "ห้องน้ำ",
    "กาแฟ",
    "น้ำผึ้ง",
    "ไข่มัน",
    "คาว",
    "ประทับใจ",
    "ราคา",
    "เกินไป",
    "รับไม่ไหว",
    "ว"
  ]
}
```

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text ```json
"keywords": ["ลาเต้", "กาแฟ", "ร้านวาวี", "อารีย์", "Starbuck", "แฟนๆ", "Starbuck", "โปรด", "test", "Taste", "อร่อย", "ราคาไม่แพง", "กินบ่อย", "จุกได้", "ถ่ายรูป", "ซากอารยธรรม", "เมพพพพ", "เมพพ", "กาแฟ", "รสชาติ", "กลิ่น", "มวยรุ่นเดียวกะ Starbuck", "การชิม", "รสชาติและกลิ่น", "เมนูอื่นๆ", "สรุป", "ถ่ายโดย", "Tan"
```



In [133]:
!ollama pull scb10x/typhoon2.1-gemma3-4b

In [134]:
!ollama list

NAME                                  ID              SIZE      MODIFIED               
scb10x/typhoon2.1-gemma3-4b:latest    8cfab9097c9d    2.6 GB    Less than a second ago    
gemma3:270m                           e7d36fb2c3b3    291 MB    16 seconds ago            


In [135]:
llm3 = OllamaLLM(model="gemma3:270m")

In [136]:
keyword_extract = keyword_prompt | llm3 | StrOutputParser()

In [137]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 59.8 ms, sys: 6.97 ms, total: 66.8 ms
Wall time: 6.66 s


In [138]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text ```json
{
  "keywords": [
    "ร้านอาหารใหญ่",
    "ห้องน้ำ",
    "กาแฟ",
    "น้ำผึ้ง",
    "การทานอาหาร",
    "ราคา",
    "เลี้ยวเข้ามา",
    "ห้องทานหลักๆ",
    "กาแฟน้ำผึ้ง",
    "แพงเวอร์",
    "อย่าสั่งเลย",
    "ลาบไข่ต้ม",
    "ไข่มันคาว",
    "ไม่ประทับใจ",
    "ทอดมันหัวปลีกรอบอร่อย",
    "เบิ้ล"
  ]
}
```

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text ```json
{
  "keywords": [
    "ลาเต้",
    "ร้อน",
    "แก้วนึง",
    "ร้อน",
    "วางอยู่ข้างหน้า",
    "เล่นลายฟองนม",
    "เว้ยยย",
    "ซากอารยธรรมลาเต้เมพๆ",
    "เมพพพพ",
    "เมพพพพ",
    "ซากอารยธรรมลาเต้เมพๆ",
    "รสชาติ 1",
    "2",
    "เครื่องดื่ม",
    "กาแฟ",
    "ร้านวาวี",
    "อารีย์",
    "สวย",
    "กินบ่อย",
    "จุกได้",
    "แหล่มไม่แพง",
    "เสียเงิน",
    "บ่อยก็จุกได้"
  ]
}
```



In [139]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class ReviewKeywords(BaseModel):
    keywords: str = Field(
        ...,  # Required field
        description="3 keywords separated by comma",
        examples=["กาแฟ, บรรยากาศ, ราคา"]
    )

class KeywordsExtraction(BaseModel):
    extracted_qualities: ReviewKeywords

In [140]:
keywords_parser = JsonOutputParser(pydantic_object=KeywordsExtraction)

In [141]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [142]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 43.3 ms, sys: 3.67 ms, total: 47 ms
Wall time: 3.97 s


In [143]:
results[0]['text']['keywords']

['ร้านอาหารใหญ่',
 'ห้องน้ำ',
 'กาแฟ',
 'น้ำผึ้ง',
 'น้ำผึ้ง',
 'ลาบไข่ต้ม',
 'ไข่มัน',
 'ขนมไทย',
 'ขนมหวาน',
 'ฟรี',
 'ราคา',
 'ไม่ไหว',
 'ไม่ประทับใจ',
 'แพงเกิน',
 'รับไม่ไหว',
 'กินแล้วมีขนมหวานให้กินฟรีเล็กน้อย']

In [144]:
results[1]['text']['keywords']

['ลาเต้',
 'ร้อน',
 'แก้วนึง',
 'เสริฟ',
 'หุๆ',
 'แมพ',
 'เมพ',
 'ซากอารยธรรมลาเต้',
 'เมพพพพ',
 'โกโก้',
 'น้ำส้มกะเค้ก',
 'แจ่ม',
 'ลากาแฟ']

In [145]:
llm3 = OllamaLLM(
    model="scb10x/typhoon2.1-gemma3-4b",
    temperature=0.05
)

keyword_extract = keyword_prompt | llm3 | keywords_parser

In [146]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 152 ms, sys: 8.45 ms, total: 161 ms
Wall time: 26.1 s


In [147]:
results[0]['text']['keywords'], results[1]['text']['keywords']

(['ราคา', 'รสชาติ', 'ขนาด'], ['กาแฟ', 'วาวี', 'อารีย์'])

In [148]:
keyword_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract keywords 3 keywords เท่านั้น ที่เป็นตัวแทนของ Review นี้ได้ดีที่สุด

    Review: {review}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'keywords'
    และค่าเป็นข้อความที่ดึงได้คั่นด้วย comma
    เช่น {{"keywords": "กาแฟ, บรรยากาศ, ราคา"}}

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [149]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [150]:
inputs = [{"review": r} for r in sample_reviews]

In [151]:
%%time
results = keyword_extract.batch(inputs)

CPU times: user 19.2 ms, sys: 584 µs, total: 19.8 ms
Wall time: 2.29 s


In [152]:
results

[{'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
 {'keywords': 'กาแฟ, บรรยากาศ, รสชาติ'}]

In [153]:
sample_reviews_1000 = coffee_reviews_df['column0'].head(1000).tolist()
inputs_1000 = [{"review": r} for r in sample_reviews_1000]

BATCH_SIZE = 8

results_1000 = []

In [154]:
from tqdm import tqdm

In [155]:
for i in tqdm(range(0, len(inputs_1000), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_1000[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_1000.append({"review": inp["review"], "text": out})

Extracting keywords: 100%|██████████| 125/125 [19:44<00:00,  9.48s/it]


In [156]:
for result in results_1000[0:10]:
    print(result['text']['keywords'])

ราคา, บรรยากาศ, ไข่ต้ม
กาแฟ, บรรยากาศ, รสชาติ
โดนัท, รสชาติ, ความหวาน
วาซาบิ, ปลาซาบะ, ข้าวหน้าปลา
โดนัท, Original Glazed, คิว
วังพญาไท, นรสิงห์, คาเฟ่
อาหารทะเล, ราคา, บรรยากาศ
กาแฟ, เค้ก, พนักงาน
ฮันนี่ โทสท์, ราคา, รสชาติ
คาเฟ่, วานิลลา, คาโบนาร่า


In [157]:
  results_1000[0]

{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
 'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'}}

In [158]:
context_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract context ใน Review ที่เป็นตัวแทนที่บรรยายคำว่า {keyword} ได้ดีที่สุด

    Review: {review}

    ตอบผลลัพธ์ในรูปแบบ JSON โดยมีคีย์เป็น 'context'
    และค่าเป็นข้อความที่ดึงได้จาก Review

    ตัวอย่าง
    keyword คือ "กาแฟ"
    context: "กาแฟที่นี่อร่อยมาก หอมกลิ่นกาแฟคั่วสด"

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [159]:
sentiment_prompt = PromptTemplate(
    template="""จากข้อความต่อไปนี้ ให้วิเคราะห์ว่าเป็น sentiment แบบไหน
    เลือกจากตัวเลือกเหล่านี้เท่านั้น: "positive", "negative", "neutral"

    ข้อความ: {message}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'sentiment'
    และค่าเป็นหนึ่งใน list ต่อไปนี้ [positive, negative, neutral]

    คำแนะนำ
    - positive คือ ข้อความที่แสดงความพอใจ ชอบ ดี
    - negative คือ ข้อความที่แสดงความไม่พอใจ ไม่ชอบ แย่
    - neutral คือ ข้อความที่ไม่แสดงความรู้สึกชัดเจน เป็นกลาง

    # ตัวอย่าง
    # ข้อความ "อร่อยมาก" → {{"sentiment": "positive"}}
    # ข้อความ "แพงไป" → {{"sentiment": "negative"}}
    # ข้อความ "ปกติ" → {{"sentiment": "neutral"}}
    """
)


In [160]:
class ContextKeyword(BaseModel):
    context: str = Field(..., description="context that best represent this keyword")

class ContexExtraction(BaseModel):
    extracted: ContextKeyword

context_parser = JsonOutputParser(pydantic_object=ContexExtraction)

In [161]:
context_extract = context_prompt | llm3 | context_parser

In [162]:
class Sentiment(BaseModel):
    sentiment: str = Field(..., description="sentiment of review")

class SentimentExtraction(BaseModel):
    extracted: Sentiment


sentiment_parser = JsonOutputParser(pydantic_object=SentimentExtraction)

In [163]:
sentiment_extract = sentiment_prompt | llm3 | sentiment_parser

In [164]:
keywords = results_1000[0]['text']['keywords']


In [165]:
context_inputs = []

for review_data in tqdm(results_1000, desc="Preparing context inputs"):
    review = review_data['review']
    keywords = review_data['text']['keywords']
    for keyword in keywords:
        context_inputs.append({"review": review, "keyword": keyword.strip()})

Preparing context inputs: 100%|██████████| 1000/1000 [00:00<00:00, 92661.08it/s]


In [166]:
len(context_inputs)

24798

In [167]:
context_results = []
for i in tqdm(range(len(results_1000)), desc="Extracting contexts"):
   result = context_extract.batch(context_inputs[i*3:(i*3)+3])
   contexts_list = [d['context'] for d in result]
   results_1000[i]['contexts'] = contexts_list

Extracting contexts: 100%|██████████| 1000/1000 [1:30:22<00:00,  5.42s/it]


In [168]:
results_1000[:2]

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
  'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
  'contexts': ['ร้านอาหารใหญ่มากกกกกกก',
   'ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี',
   'ห้องทานอาหารอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย']},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสว

In [169]:
sentiment_inputs = []

for review_data in tqdm(results_1000, desc="Preparing sentiment inputs"):
    contexts = review_data['contexts']
    for context in contexts:
        sentiment_inputs.append({"message": context})

Preparing sentiment inputs: 100%|██████████| 1000/1000 [00:00<00:00, 331749.11it/s]


In [170]:
len(sentiment_inputs)

3000

In [171]:
sentiment_inputs[:6]

[{'message': 'ร้านอาหารใหญ่มากกกกกกก'},
 {'message': 'ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี'},
 {'message': 'ห้องทานอาหารอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย'},
 {'message': 'ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี'},
 {'message': 'ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว'},
 {'message': 'ทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล'}]

In [174]:
for i in tqdm(range(len(results_1000)), desc="Extracting sentiment"):
   result = sentiment_extract.batch(sentiment_inputs[i*3:(i*3)+3])
   sentiment_list = [d['sentiment'] for d in result]
   results_1000[i]['sentiment'] = sentiment_list

Extracting sentiment: 100%|██████████| 1000/1000 [43:42<00:00,  2.62s/it]


In [175]:
results_1000[0:2]

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
  'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
  'contexts': ['ร้านอาหารใหญ่มากกกกกกก',
   'ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี',
   'ห้องทานอาหารอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย'],
  'sentiment': ['neutral', 'neutral', 'negative']},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรยา

In [176]:
import pandas as pd

df = pd.DataFrame([
    {
        'review': r['review'],
        **{f'keyword{i+1}': k.strip() for i, k in enumerate(r['text']['keywords'])},
        **{f'context{i+1}_keyword': c for i, c in enumerate(r['contexts'])},
        **{f'sentiment{i+1}': s for i, s in enumerate(r['sentiment'])}
    }
    for r in results_1000
])

In [177]:
df.to_csv('results.csv', index=False, encoding='utf-8-sig')